In [1]:
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import HuberRegressor, LinearRegression, RANSACRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from pykrige.ok import OrdinaryKriging
import mapbox
import plotly.graph_objs as go
import plotly.express as px
from global_land_mask import globe
import statsmodels.api as sm
import random

In [2]:
ds_chl = xr.open_dataset('C:/Users/Acer/Documents/SchoolHard/Thesis/Code/dataset//chl_merged_2002_08_2022_10.nc')

In [3]:
ds_chl

<xarray.Dataset>
Dimensions:  (time: 243, lat: 51, lon: 33, rgb: 3, eightbitcolor: 256)
Coordinates:
  * lat      (lat) float32 7.729 7.687 7.646 7.604 ... 5.771 5.729 5.687 5.646
  * lon      (lon) float32 125.2 125.2 125.3 125.3 ... 126.4 126.4 126.5 126.5
  * time     (time) datetime64[ns] 2002-08-01 2002-09-01 ... 2022-10-01
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    chlor_a  (time, lat, lon) float32 ...
    palette  (time, rgb, eightbitcolor) uint8 ...
Attributes: (12/62)
    product_name:                     AQUA_MODIS.20020801_20020831.L3m.MO.CHL...
    instrument:                       MODIS
    title:                            MODISA Level-3 Standard Mapped Image
    project:                          Ocean Biology Processing Group (NASA/GS...
    platform:                         Aqua
    source:                           satellite observations from MODIS-Aqua
    ...                               ...
    cdm_data_type:                    grid
    keywords:                         Earth Science > Oceans > Ocean Chemistr...
    keywords_vocabulary:              NASA Global Change Master Directory (GC...
    data_bins:                        17371971
    data_minimum:                     0.0031615312
    data_maximum:                     86.2636

In [4]:
#define time steps, 243 monthly observations
start_date = pd.Timestamp('2002-08-01')
end_date = pd.Timestamp('2022-10-01')

#Create a new time coordinate that represents the month and year
time_coords = pd.date_range(start=start_date, end=end_date, freq='MS')

In [5]:
n = 24
samples = random.sample(list(time_coords), n)

In [6]:
#Define the number of folds for cross validation
n_splits = 3

In [7]:
#Create a KFold object to split the data
kf = KFold(n_splits=n_splits)

In [8]:
#metrics list 
mse_list = [] 
rmse_list = [] 
mae_list = []

In [9]:
for t in range(len(samples)):
    
    print('set: ' + str(t))
    
    monthly_data = ds_chl['chlor_a'].sel(time= samples[t])
    lon = monthly_data['lon'].values
    lat = monthly_data['lat'].values
    
    #Convert the dataset to a pandas dataframe
    df_chl = monthly_data.to_dataframe().reset_index()
    
    #convert dataframe to geodataframe 
    gdf_chl = gpd.GeoDataFrame(df_chl, geometry=gpd.points_from_xy(df_chl.lon, df_chl.lat))
    
    #SET CRS to WGS84
    gdf_chl.crs = 'WGS84'
    
    #dropna in gdf
    gdf_chl = gdf_chl.dropna()
    
    # Calculate winsorized values
    winsorized = np.clip(gdf_chl['chlor_a'], gdf_chl['chlor_a'].quantile(0.05), gdf_chl['chlor_a'].quantile(0.95))

    # Define Huber loss function
    def huber_loss(residuals, c=1.345):
        return np.where(abs(residuals) < c, 0.5 * residuals ** 2, c * (abs(residuals) - 0.5 * c))

    # Define M-estimator function
    def m_estimator(data, loss_function, tuning_param):
        # Add a constant column to serve as the intercept
        exog = sm.add_constant(data)
        model = sm.RLM(gdf_chl['chlor_a'], exog=exog, M=sm.robust.norms.HuberT(t=tuning_param))
        results = model.fit()
        return results.fittedvalues

    # Apply M-estimator function to winsorized data
    final_values = m_estimator(winsorized, huber_loss, 1.345)

    # Add final values as a new column to your original GeoDataFrame
    gdf_chl['final_values'] = final_values

    # Convert the 'final_values' column to float data type if necessary
    gdf_chl['final_values'] = gdf_chl['final_values'].astype(float)
    
    #Get x, y, z values - this is the lon, lat, and variable/sst
    x = gdf_chl.geometry.x
    y = gdf_chl.geometry.y
    z = gdf_chl['final_values']
    
    x_grid = np.linspace(lon.min(), lon.max(), num=100)
    y_grid = np.linspace(lat.min(), lat.max(), num=100)
    XI, YI = np.meshgrid(x_grid, y_grid)
    
    for train_index, test_index in kf.split(gdf_chl):
        #Train and Test splits
        train_gdf = gdf_chl.iloc[train_index].reset_index()
        test_gdf = gdf_chl.iloc[test_index].reset_index()

        #Train data x,y,z
        x_train = train_gdf.geometry.x
        y_train = train_gdf.geometry.y
        z_train = train_gdf['final_values']

        #Ordinary Kriging
        orkrig = OrdinaryKriging(x_train, y_train, z_train, variogram_model="linear", verbose=False, enable_plotting=False)
    
        #Train data x,y,z
        x_test = test_gdf.geometry.x
        y_test = test_gdf.geometry.y
        z_test = test_gdf['final_values']

        ZI_test = np.zeros_like(x_test)
        for i in range(len(x_test)):
            ZI_test[i], sigma = orkrig.execute("grid", x_test[i], y_test[i])

        #MSE -mean squared error
        mse = mean_squared_error(z_test, ZI_test)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(z_test, ZI_test)
    
        mse_list.append(mse)
        rmse_list.append(rmse)
        mae_list.append(mae)

set: 0


C:\Users\Acer\anaconda3\lib\site-packages\statsmodels\robust\robust_linear_model.py:288: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  warnings.warn('Estimated scale is 0.0 indicating that the most'


set: 1


C:\Users\Acer\anaconda3\lib\site-packages\statsmodels\robust\robust_linear_model.py:288: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  warnings.warn('Estimated scale is 0.0 indicating that the most'


KeyboardInterrupt: 

In [40]:
# Compute the mean  of MSE MAE RMSE across all folds
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

mean_rmse = np.mean(rmse_list)
std_rmse = np.std(rmse_list)

mean_mae = np.mean(mae_list)
std_mae = np.std(mae_list)

print(mean_mse)
print(mean_rmse)
print(mean_mae)

0.30735701464414417
0.25018991340411184
0.14282863883086286


In [38]:
gdf_chl

,lat,lon,time,chlor_a,geometry,final_values
310,7.354164,125.729179,2014-08-01,1.160169,POINT (125.72918 7.35416),0.760520
311,7.354164,125.770844,2014-08-01,1.805594,POINT (125.77084 7.35416),0.760520
312,7.354164,125.812508,2014-08-01,3.383218,POINT (125.81251 7.35416),0.760520
313,7.354164,125.854179,2014-08-01,3.383274,POINT (125.85418 7.35416),0.760520
342,7.312498,125.687508,2014-08-01,2.592484,POINT (125.68751 7.31250),0.760520
...,...,...,...,...,...,...
1678,5.645831,126.354179,2014-08-01,0.037992,POINT (126.35418 5.64583),0.037992
1679,5.645831,126.395844,2014-08-01,0.039323,POINT (126.39584 5.64583),0.039323
1680,5.645831,126.437508,2014-08-01,0.036374,POINT (126.43751 5.64583),0.037583
1681,5.645831,126.479179,2014-08-01,0.036533,POINT (126.47918 5.64583),0.037583


In [32]:
# Test for raw chl data
for t in range(len(samples)):
    
    print('set: ' + str(t))
    
    monthly_data = ds_chl['chlor_a'].sel(time= samples[t])
    lon = monthly_data['lon'].values
    lat = monthly_data['lat'].values
    
    #Convert the dataset to a pandas dataframe
    df_chl = monthly_data.to_dataframe().reset_index()
    
    #convert dataframe to geodataframe 
    gdf_chl = gpd.GeoDataFrame(df_chl, geometry=gpd.points_from_xy(df_chl.lon, df_chl.lat))
    
    #SET CRS to WGS84
    gdf_chl.crs = 'WGS84'
    
    #dropna in gdf
    gdf_chl = gdf_chl.dropna()
    
    #Get x, y, z values - this is the lon, lat, and variable/sst
    x = gdf_chl.geometry.x
    y = gdf_chl.geometry.y
    z = gdf_chl['chlor_a']
    
    x_grid = np.linspace(lon.min(), lon.max(), num=100)
    y_grid = np.linspace(lat.min(), lat.max(), num=100)
    XI, YI = np.meshgrid(x_grid, y_grid)
    
    for train_index, test_index in kf.split(gdf_chl):
        #Train and Test splits
        train_gdf = gdf_chl.iloc[train_index].reset_index()
        test_gdf = gdf_chl.iloc[test_index].reset_index()

        #Train data x,y,z
        x_train = train_gdf.geometry.x
        y_train = train_gdf.geometry.y
        z_train = train_gdf['chlor_a']

        #Ordinary Kriging
        orkrig = OrdinaryKriging(x_train, y_train, z_train, variogram_model="linear", verbose=False, enable_plotting=False)
    
        #Train data x,y,z
        x_test = test_gdf.geometry.x
        y_test = test_gdf.geometry.y
        z_test = test_gdf['chlor_a']

        ZI_test = np.zeros_like(x_test)
        for i in range(len(x_test)):
            ZI_test[i], sigma = orkrig.execute("grid", x_test[i], y_test[i])

        #MSE -mean squared error
        mse = mean_squared_error(z_test, ZI_test)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(z_test, ZI_test)
    
        mse_list.append(mse)
        rmse_list.append(rmse)
        mae_list.append(mae)

set: 0
set: 1
set: 2
set: 3
set: 4
set: 5
set: 6
set: 7
set: 8
set: 9
set: 10
set: 11
set: 12
set: 13
set: 14
set: 15
set: 16
set: 17
set: 18
set: 19
set: 20
set: 21
set: 22
set: 23


In [33]:
# Compute the mean  of MSE MAE RMSE across all folds
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

mean_rmse = np.mean(rmse_list)
std_rmse = np.std(rmse_list)

mean_mae = np.mean(mae_list)
std_mae = np.std(mae_list)

print(mean_mse)
print(mean_rmse)
print(mean_mae)

0.43940268289046697
0.3001658603894449
0.16696281852258377


In [35]:
gdf_chl

,lat,lon,time,chlor_a,geometry
310,7.354164,125.729179,2014-08-01,1.160169,POINT (125.72918 7.35416)
311,7.354164,125.770844,2014-08-01,1.805594,POINT (125.77084 7.35416)
312,7.354164,125.812508,2014-08-01,3.383218,POINT (125.81251 7.35416)
313,7.354164,125.854179,2014-08-01,3.383274,POINT (125.85418 7.35416)
342,7.312498,125.687508,2014-08-01,2.592484,POINT (125.68751 7.31250)
...,...,...,...,...,...
1678,5.645831,126.354179,2014-08-01,0.037992,POINT (126.35418 5.64583)
1679,5.645831,126.395844,2014-08-01,0.039323,POINT (126.39584 5.64583)
1680,5.645831,126.437508,2014-08-01,0.036374,POINT (126.43751 5.64583)
1681,5.645831,126.479179,2014-08-01,0.036533,POINT (126.47918 5.64583)


In [39]:
print(pd.concat([gdf_chl['chlor_a'].describe(), gdf_chl['final_values'].describe()], axis=1))

          chlor_a  final_values
count  889.000000    889.000000
mean     0.220976      0.184545
std      0.358942      0.188920
min      0.033760      0.037583
25%      0.075356      0.075356
50%      0.116736      0.116736
75%      0.198809      0.198809
max      3.383274      0.760520


In [ ]:
# raw chl test data
0.43940268289046697
0.3001658603894449
0.16696281852258377

# winsorized and m-estimated CHL test data
0.30735701464414417
0.25018991340411184
0.14282863883086286


            chlor_a  chl_final
count  889.000000    889.000000
mean     0.220976      0.184545
std      0.358942      0.188920
min      0.033760      0.037583
25%      0.075356      0.075356
50%      0.116736      0.116736
75%      0.198809      0.198809
max      3.383274      0.760520